In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [5]:
btc_usd = pd.read_csv("BTC-USD.csv").loc[:,"Close"]
btc_cad = pd.read_csv("BTC-CAD.csv").loc[:,""]
btc_eur = pd.read_csv("BTC-EUR.csv")
btc_gbp = pd.read_csv("BTC-GBP.csv")
eur_cad = pd.read_csv("EUR-CAD.csv")
eur_gbp = pd.read_csv("EUR-GBP.csv")
eur_usd = pd.read_csv("EUR-USD.csv")
gbp_cad = pd.read_csv("GBP-CAD.csv")
gbp_usd = pd.read_csv("GBP-USD.csv")
cad_usd = pd.read_csv("CAD-USD.csv")

In [6]:
btc_usd.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-11-29,17719.634766,18283.628906,17559.117188,18177.484375,18177.484375,31133957704
1,2020-11-30,18178.322266,19749.263672,18178.322266,19625.835938,19625.835938,47728480399
2,2020-12-01,19633.769531,19845.974609,18321.921875,18802.998047,18802.998047,49633658712
3,2020-12-02,18801.744141,19308.330078,18347.718750,19201.091797,19201.091797,37387697139
4,2020-12-03,19205.925781,19566.191406,18925.785156,19445.398438,19445.398438,31930317405
